<a href="https://colab.research.google.com/github/mcash8/DVBS2X-DNN-Implementation/blob/main/FFNNEq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import scipy.io as spio
import math
import time
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

start_time = time.time()

num_classes = 2
batch_size = 64
epochs = 100
BOs = ["1", "7", "30"] #backoffs

for bo in BOs: 
    # Load the data

    matname = "Data_BO" + bo + ".mat"
    print(matname)
    
    mat = spio.loadmat(matname, squeeze_me=True)
    x_train = mat['x_train']
    x_valid = mat['data_val']
    x_test = mat['x_pred']
    y_train = mat['y_train']
    y_valid = mat['target_val']
    #y_test = mat['y_test']


    # Convert the data to floats between 0 and 1.
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    x_test = x_test.astype('float32')

    print(x_train.shape, 'train samples')
    print(x_valid.shape, 'valid samples')
    #print(x_test.shape, 'test samples')
    print(y_train.shape, 'train labels')
    print(y_valid.shape, 'valid labels')
    #print(y_test.shape, 'test labels')
    print('Label Examples:\n', y_train[0:9]);

    # Formatting
    fmtLen = int(math.ceil(math.log(max(batch_size, y_valid.shape[0]),10)))

    # Define the network
    model = Sequential()
    model.add(Dense(50, activation='tanh', input_dim=18))
    model.add(Dense(50, activation='tanh'))
    model.add(Dense(num_classes, activation='linear'))

    model.summary()

    model.compile(loss=keras.metrics.mean_squared_error,
                optimizer=SGD(),
                metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=2,
                        validation_data=(x_valid, y_valid))

    score = model.evaluate(x_train, y_train, verbose=2)
    print('Final Training MSE:', score[0])
    print('Final Training RMSE:', score[1])

    score = model.evaluate(x_valid, y_valid, verbose=2)
    print('Final Validation MSE:', score[0])
    print('Final Validation RMSE:', score[1])

    #score = model.evaluate(x_test, y_test, verbose=1)
    #print('Test MSE:', score[0])
    #print('Test RMSE:', score[1])


    predictions = model.predict(x_test)
    matname = "predictionsBO" + bo + ".mat"
    spio.savemat(matname, {'pred': predictions})
    savename = "deep_model_BO" + bo + ".h5"
    model.save(savename)


    elapsedTime = (time.time() - start_time)/60
    print("--- %.2f minutes ---" % elapsedTime)

    '''
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    figname = 'lossPlot'+snr+'.png'
    plt.savefig(figname)
    '''

Data_BO1.mat
(101500, 18) train samples
(1015, 18) valid samples
(101500, 2) train labels
(1015, 2) valid labels
Label Examples:
 [[ 0.22448 -0.22448]
 [-0.22448  0.22448]
 [-0.70711  0.70711]
 [-0.22448  0.22448]
 [-0.25882  0.96593]
 [ 0.70711 -0.70711]
 [ 0.96593  0.25882]
 [ 0.25882  0.96593]
 [-0.22448 -0.22448]]
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 50)                950       
                                                                 
 dense_25 (Dense)            (None, 50)                2550      
                                                                 
 dense_26 (Dense)            (None, 2)                 102       
                                                                 
Total params: 3,602
Trainable params: 3,602
Non-trainable params: 0
______________________________________________________________